<a href="https://colab.research.google.com/github/binit-13/Machine-Learning-Notes/blob/main/Encoder_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import tensorflow
from tensorflow import keras
from keras.layers import Dense, Input, LSTM
from keras.models import Model
from keras.utils import Sequence

In [ ]:
print(tensorflow.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


1. Text Preprocessing
2. Text Vectorization --> OHE/Bag of Words/TF-IFD/Embeddings

In [ ]:
num_sam = 227816
input_text = []
output_text = []
input_char = set()
output_char = set()

with open('/content/fra.txt','r',encoding='utf-8') as f:
  lines = f.read().split('\n')
  print(len(lines))

  for i in range(0, 210000,1000):
    for line in lines[i:i+1000]:
      input, output, _ = line.split('\t')
      output = '\t' + output + '\n'
      input_text.append(input)
      output_text.append(output)

  for chars in input_text:
    for char in chars:
      if char not in input_char:
        input_char.add(char)

  for chars in output_text:
    for char in chars:
      if char not in output_char:
        output_char.add(char)

input_char = sorted(list(input_char))
output_char = sorted(list(output_char))


227816


In [ ]:
print(input_text[1])
print(output_text[0])

Go.
	Va !



In [ ]:
output_text[50000]

'\tJe viens juste de commencer.\n'

In [ ]:
encoder_token = len(input_char)
decoder_token = len(output_char)
max_encoder_seq = max([len(text) for text in input_text])
max_decoder_seq = max([len(text) for text in output_text])

In [ ]:
print(encoder_token) ##--> total alphabets and symbols
print(decoder_token)
print(max_encoder_seq) ## --> max length of sentence in whole document
print(max_decoder_seq)

88
112
48
100


In [ ]:
## OHE
input_token_index = {char: i for i, char in enumerate(input_char)}
output_token_index = {char: i for i, char in enumerate(output_char)}

encoder_input_data = np.zeros((len(input_text),max_encoder_seq, encoder_token),dtype='float16')
decoder_input_data = np.zeros((len(input_text),max_decoder_seq, decoder_token),dtype='float16')
decoder_output_data = np.zeros((len(input_text),max_decoder_seq, decoder_token),dtype='float16')

total_num_rows = [i for i in range(0,210000)]

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '/': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '?': 23,
 'A': 24,
 'B': 25,
 'C': 26,
 'D': 27,
 'E': 28,
 'F': 29,
 'G': 30,
 'H': 31,
 'I': 32,
 'J': 33,
 'K': 34,
 'L': 35,
 'M': 36,
 'N': 37,
 'O': 38,
 'P': 39,
 'Q': 40,
 'R': 41,
 'S': 42,
 'T': 43,
 'U': 44,
 'V': 45,
 'W': 46,
 'X': 47,
 'Y': 48,
 'Z': 49,
 'a': 50,
 'b': 51,
 'c': 52,
 'd': 53,
 'e': 54,
 'f': 55,
 'g': 56,
 'h': 57,
 'i': 58,
 'j': 59,
 'k': 60,
 'l': 61,
 'm': 62,
 'n': 63,
 'o': 64,
 'p': 65,
 'q': 66,
 'r': 67,
 's': 68,
 't': 69,
 'u': 70,
 'v': 71,
 'w': 72,
 'x': 73,
 'y': 74,
 'z': 75,
 '\xa0': 76,
 '\xad': 77,
 '°': 78,
 'º': 79,
 'é': 80,
 'ï': 81,
 'ú': 82,
 '–': 83,
 '‘': 84,
 '’': 85,
 '₂': 86,
 '€': 87}

In [ ]:
for num in range(0,210000,1000):
  for i, inp, out in zip(total_num_rows[num:num+1000],input_text[num:num+1000], output_text[num:num+1000]):
    for t, char in enumerate(inp):
      encoder_input_data[i,t,input_token_index[char]] = 1.0
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1.0 #padding

    for t, char in enumerate(out):
      decoder_input_data[i,t,output_token_index[char]]=1.0
      if t > 0:
        decoder_output_data[i, t-1, output_token_index[char]]=1
    decoder_input_data[i,t+1:,output_token_index[' ']] = 1.0 ##padding
    decoder_output_data[i,t:,output_token_index[' ']] = 1.0 ##padding

In [ ]:
encoder_input_data[]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float16)

In [ ]:
np.set_printoptions(threshold = np.inf)
encoder_input_data[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [ ]:
encoder_input = Input(shape=(None, encoder_token))
encoder = LSTM(256,return_state=True)
enco_output, h_state, c_state = encoder(encoder_input)

encoder_state = [h_state, c_state]

decoder_input = Input(shape=(None, decoder_token))
decoder = LSTM(256, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder(decoder_input, initial_state=encoder_state)
decoder_dense = Dense(decoder_token, activation='softmax')
decoder_output = decoder_dense(decoder_output)

model = Model([encoder_input,decoder_input],decoder_input)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

batch_size = 32
total_samples = 10000

for i in range(0, total_samples, batch_size):
  start_idx = i
  end_idx = min(i + batch_size, total_samples)
  model.fit([encoder_input_data[start_idx:end_idx], decoder_input_data[start_idx:end_idx]],
      decoder_output_data[start_idx:end_idx],
      batch_size=batch_size,
      epochs=10,
      validation_split=0.2)

In [ ]:
import tracemalloc
from time import sleep
import matplotlib.pyplot as plt

# sampling time in seconds
SAMPLING_TIME = 0.001

class MemoryMonitor:
    def __init__(self, close=True):

        # start tracemalloc and sets
        # measurement atribute to True
        tracemalloc.start()
        self.keep_measuring = True
        self.close = close ##True

    def measure_usage(self):

        """
        Takes measurements of used memory on
        regular intevals determined by the
        global SAMPLING_TIME constant
        """

        # list to store memory usage samples
        usage_list = []

        # keeps going until someone changes this parameter to false
        while self.keep_measuring:

            # takes a sample, stores it in the usage_list and sleeps
            current, peak = tracemalloc.get_traced_memory()
            usage_list.append(current/1e6)
            sleep(SAMPLING_TIME)

        # stop tracemalloc and returns list
        if self.close:
            tracemalloc.stop()
        return usage_list

# imports executor
from concurrent.futures import ThreadPoolExecutor
from functools import wraps

def plot_memory_use(history, fn_name, open_figure=True, offset=0, **kwargs):

    """Function to plot memory use from a history collected
        by the MemoryMonitor class
    """

    # getting times from counts and sampling time
    times = (offset + np.arange(len(history))) * SAMPLING_TIME

    # opening figure and plotting
    if open_figure:
        plt.figure(figsize=(10,3), dpi=120)
    plt.plot(times, history, 'k--', linewidth=1)
    plt.fill_between(times, history, alpha=0.5, **kwargs)

    # axes titles
    plt.ylabel('Memory usage [MB]')
    plt.xlabel('Time [seconds]')
    plt.title(f'{fn_name} memory usage over time')

    # legend
    plt.legend();

def track_memory_use(plot=True, close=True, return_history=False):

    def meta_wrapper(fn):

        """
        This function is meant to be used as a decorator
        that informs wrapped function memory usage
        """

        # decorator so we can retrieve original fn
        @wraps(fn)
        def wrapper(*args, **kwargs):

            """
            Starts wrapped function and holds a process
            to sample memory usage while executing it
            """

            # context manager for executor
            with ThreadPoolExecutor() as executor:

                # start memory monitor
                monitor = MemoryMonitor(close=close)
                mem_thread = executor.submit(monitor.measure_usage)

                # start wrapped function and get its result
                try:
                    fn_thread = executor.submit(fn, *args, **kwargs)
                    fn_result = fn_thread.result()

                # when wrapped function ends, stop measuring
                finally:
                    monitor.keep_measuring = False
                    history = mem_thread.result()

                # inform results via prints and plot
                print(f'Current memory usage: {history[-1]:2f}')
                print(f'Peak memory usage: {max(history):2f}')
                if plot:
                    plot_memory_use(history, fn.__name__)
            if return_history:
                return fn_result, history
            else:
                return fn_result

        return wrapper

    return meta_wrapper